# Assignment

In [82]:
import pandas as pd
from pandas_schema import Column, Schema
from pandas_schema.validation import CanConvertValidation
from GeoBases import GeoBase

filename = '/media/sf_SharedVirtbox/bookings.csv/bookings.csv'
columns = ['arr_port', 'pax']

bookings_df = pd.read_csv(filename, nrows=10, sep='^', usecols=columns)

## Explore data

In [86]:
bookings_df.head()

,arr_port,pax
0,LHR,-1
1,CLT,1
2,CLT,1
3,SVO,1
4,SVO,1


## Validate some CSV entries & process dataframes

In [90]:
# Note: Sum of pax returned float -> invalid entry in dataset -> verification needed
# Choosing to remove the invalid entries
# Very basic validation

schema = Schema([
    Column('arr_port', [CanConvertValidation(str)]),
    Column('pax', [CanConvertValidation(int)])
])

def removeInvalidEntries(chunk, errors):
    for error in errors:
        print(error.__str__() + "\nRemoving this row\n")
        chunk = chunk.drop([error.row], axis=0)
    return chunk

In [91]:
# Read huge dataset (GBs size .csv) by chunks to avoid OOM, chunksize at will for now 
# Perform groupby on chunked dataframes, then a final one combining these
chunksize = 100000

dataframes = []
df = None

for chunk in pd.read_csv(filename, sep='^', usecols=columns, chunksize=chunksize):
    errors_in_chunk = schema.validate(chunk)
    if errors_in_chunk:
        chunk = removeInvalidEntries(chunk, errors_in_chunk)
    dataframes.append(chunk.groupby(['arr_port'], as_index=False).sum())

df = pd.concat(dataframes)
nb_pax_per_airport = df.groupby(['arr_port'], as_index=False).sum()

{row: 5000007, column: "pax"}: "nan" cannot be converted to type <class 'int'>
Removing this row



## Show list of 10 biggest airports per pax arrivals

In [92]:
nb_pax_per_airport_10 = nb_pax_per_airport.sort_values('pax', ascending=False).head(10)

In [102]:
nb_pax_per_airport_10

,arr_port,pax
1088,LHR,88809.0
1190,MCO,70930.0
1050,LAX,70530.0
1047,LAS,69630.0
886,JFK,66270.0
315,CDG,64490.0
216,BKK,59460.0
1228,MIA,58150.0
1719,SFO,58000.0
517,DXB,55590.0


## Show the same list with readable names

In [106]:
# Load database of airports from Geobase
geo_a = GeoBase(data='airports', verbose=False)

def getAirportName(iata_code):
    return geo_a.get(iata_code.strip(), 'name')

In [106]:
nb_pax_per_airport_10['arr_port'] = nb_pax_per_airport_10['arr_port'].apply(getAirportName)

In [108]:
nb_pax_per_airport_10

,arr_port,pax
1088,London Heathrow Airport,88809.0
1190,Orlando International Airport,70930.0
1050,Los Angeles International Airport,70530.0
1047,McCarran International Airport,69630.0
886,John F Kennedy International Airport,66270.0
315,Paris - Charles-de-Gaulle,64490.0
216,Suvarnabhumi,59460.0
1228,Miami International Airport,58150.0
1719,San Francisco International Airport,58000.0
517,Dubai International Airport,55590.0
